<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_timm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cpu


In [4]:
# search file paths
from glob import glob


img_paths_train = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png') # *: all
img_paths_train = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png') # *: all
img_paths_val = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/val_50/*.png')

In [5]:
# number of images
len(img_paths_train), len(img_paths_val)

(22500, 2500)

In [6]:
import torch
from PIL import Image
import numpy as np


class PretrainDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = Image.open(path).convert('RGB') # read img

        # transform img
        img = self.transform(img)

        # Read class index
        cls_idx = int((path.split('/')[-1]).split('_')[0])
        cls_idx = torch.tensor(cls_idx, dtype=torch.int64)

        return img, cls_idx

#### Preprocess Transform

In [7]:
from torchvision import transforms
from timm.data.mixup import Mixup


# Preprocess Transform
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),
])

transform_valid = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),
])

mixup = Mixup(
    mixup_alpha=0.2, # 較小的值導致更多的原始樣本信息保留
    cutmix_alpha=1.0,
    cutmix_minmax=None,
    prob=1, # 每個樣本都會被增強機率
    switch_prob=0.5,
    mode='batch',
    label_smoothing=0.1,
    num_classes=50,
)

#### Build Dataset

In [ ]:
train_ds = PretrainDataset(img_paths_train, transform_train)
val_ds = PretrainDataset(img_paths_val, transform_valid)

In [ ]:
transform_train

ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
# Build DataLoaders
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE, shuffle=False, num_workers=1, pin_memory=True)

#### Build Model (https://pytorch.org/vision/stable/models.html#classification)

In [ ]:
from torchvision import models

model = models.efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1)
print(model)

In [ ]:
# 获取最后一层线性层的子模块列表
last_layer = list(model.children())[-1]

# 获取线性层中的权重和偏差
weights = last_layer[1].weight
bias = last_layer[1].bias

# 创建一个新的线性层，将out_features设置为50
new_linear_layer = torch.nn.Linear(in_features=weights.shape[1], out_features=50, bias=True)

# 将新的线性层的权重和偏差设置为旧线性层的权重和偏差
new_linear_layer.weight.data = weights
new_linear_layer.bias.data = bias

# 将新的线性层替换为模型的最后一层
last_layer[1] = new_linear_layer

In [ ]:
from torchsummary import summary


resize_size = transform.resize_size[0]
summary(model.to(device), (3, resize_size, resize_size))

#### Train Model

In [ ]:
from tqdm.auto import tqdm # (optional) progress bar


def training(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode. (The Dropout layer has a different behavior between the training mode and the evaluation mode.)
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True) # move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Optimization by gradients
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def testing(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [ ]:
import matplotlib.pyplot as plt
from torch import nn

def train(pretrained, freeze=False):
    print(f"pretrained weights: {pretrained}, freeze: {freeze}")
    # Model
    if pretrained:
        model = models.efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1)
    else:
        model = models.efficientnet_v2_m(weights=None)

    if freeze:
        # Freeze model
        for param in model.parameters():
            param.requires_grad = False

    # Replace classifier
    last_layer = list(model.children())[-1]
    weights = last_layer[1].weight
    bias = last_layer[1].bias
    new_linear_layer = torch.nn.Linear(in_features=weights.shape[1], out_features=50, bias=True)
    new_linear_layer.weight.data = weights
    new_linear_layer.bias.data = bias
    last_layer[1] = new_linear_layer

    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    # lower learning rate for finetuning
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=4e-3)
    # schedule = ReduceLROnplateau(optimizer, 'min')

    EPOCHS = 30
    logs = {
        'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
    }
    for epoch in tqdm(range(EPOCHS)):
        train_loss, train_acc = training(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc = testing(val_loader, model, loss_fn)

        # # Note that step should be called after test_epoch()
        # schedule.step(val_loss)

        print(f'EPOCH: {epoch:04d} \
        train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
        val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

        logs['train_loss'].append(train_loss)
        logs['train_acc'].append(train_acc)
        logs['val_loss'].append(val_loss)
        logs['val_acc'].append(val_acc)


    # plot result
    plt.figure(figsize=(6, 3))
    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.plot(logs['train_loss'])
    plt.plot(logs['val_loss'])
    plt.subplot(1, 2, 2)
    plt.title('Acc.')
    plt.plot(logs['train_acc'])
    plt.plot(logs['val_acc'])
    plt.show()

    # Save model
    torch.save(model.state_dict(), "last.pth")

#### Transfer Learning

In [ ]:
# train(pretrained=True)
train(pretrained=True, freeze=True)

pretrained weights: True, freeze: True


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: ignored